# Hybrid Recommendation using
a) Collaborative Filtering Model

b) Content-based Model

In [1]:
import pandas as pd
from random import randint

def generate_data(n_books = 3000, n_genres = 10, n_authors = 450, n_publishers = 50, n_readers = 30000, dataset_size = 100000):
    
    d = pd.DataFrame(
        {
            'book_id' : [randint(1, n_books) for _ in range(dataset_size)],
            'author_id' : [randint(1, n_authors) for _ in range(dataset_size)],
            'book_genre' : [randint(1, n_genres) for _ in range(dataset_size)],
            'reader_id' : [randint(1, n_readers) for _ in range(dataset_size)],
            'num_pages' : [randint(75, 700) for _ in range(dataset_size)],
            'book_rating' : [randint(1, 10) for _ in range(dataset_size)],
            'publisher_id' : [randint(1, n_publishers) for _ in range(dataset_size)],
            'publish_year' : [randint(2000, 2021) for _ in range(dataset_size)],
            'book_price' : [randint(1, 200) for _ in range(dataset_size)],
            'text_lang' : [randint(1,7) for _ in range(dataset_size)]
        }
    ).drop_duplicates()
    return d
  
df = generate_data(dataset_size = 100000)
df.to_csv('data.csv', index = False)
df.head()

,book_id,author_id,book_genre,reader_id,num_pages,book_rating,publisher_id,publish_year,book_price,text_lang
0,476,430,1,21991,104,5,39,2015,58,6
1,214,173,5,17336,199,2,32,2009,157,2
2,908,179,9,11370,196,1,14,2017,76,4
3,749,323,2,9639,197,2,13,2021,180,7
4,565,141,2,9715,542,6,41,2004,178,4


In [2]:
df = df.sort_values(by=['book_id'], ascending = True)

In [3]:
df.head()

,book_id,author_id,book_genre,reader_id,num_pages,book_rating,publisher_id,publish_year,book_price,text_lang
37096,1,10,7,25885,577,9,14,2007,163,3
27738,1,25,9,23452,308,6,36,2008,72,2
27851,1,226,4,17207,166,3,26,2002,42,2
15639,1,194,7,11178,625,2,30,2009,28,3
65393,1,380,4,14465,152,1,1,2005,127,5


In [4]:
# normalizing the data to 0s and 1s
import numpy as np
'''
FORMULA
X(NORMALIZED) = (X - Xminimum)/(Xmaximum - Xminimum)

 min_val = min(data)
    if min_val < 0:
        data = [x + abs(min_val) for x in data]
    max_val = max(data)
    
    return [x/max_val for x in data]

'''
def normalize(data):    
    return (data - np.min(data)/ (np.max(data)- np.min(data)))

In [5]:
#performing one hot encoding
def encoding(df, column):
    new = pd.get_dummies(df[column])
    new.reset_index(drop = True, inplace = True)
    return pd.concat([df, new], axis = 1)

In [6]:
''' 
Creating a Conten based recommendation syatem as the base system

'''

class Recommender():
    def __init__(self,df):
        self.df = df
    
    def similarity(self, v1, v2):
        return np.sum(np.dot(v1,v2)/ np.cross(v1,v2)) 
    
    def recommend(self, book_id, rec):
        ip = self.df.loc[book_id].values
        self.df['sim'] = self.df.apply(lambda x: self.similarity(ip, x.values), axis =1)
        
        return self.df.nlargest(columns = 'sim', n = rec)

In [7]:
df['num_pages_norm'] = normalize(df['num_pages'].values)
df['book_rating_norm'] = normalize(df['book_rating'].values)
df['book_price_norm'] = normalize(df['book_price'].values)
df.head()

,book_id,author_id,book_genre,reader_id,num_pages,book_rating,publisher_id,publish_year,book_price,text_lang,num_pages_norm,book_rating_norm,book_price_norm
37096,1,10,7,25885,577,9,14,2007,163,3,576.88,8.888889,162.994975
27738,1,25,9,23452,308,6,36,2008,72,2,307.88,5.888889,71.994975
27851,1,226,4,17207,166,3,26,2002,42,2,165.88,2.888889,41.994975
15639,1,194,7,11178,625,2,30,2009,28,3,624.88,1.888889,27.994975
65393,1,380,4,14465,152,1,1,2005,127,5,151.88,0.888889,126.994975


In [8]:
df = encoding(df = df, column = 'publish_year')
df = encoding(df = df, column = 'book_genre')
df = encoding(df = df, column = 'text_lang')

#dropping redndant columns
cols = ['publish_year', 'book_genre', 'num_pages', 'book_rating', 'book_price', 'text_lang']
df.drop(columns = cols, inplace = True)

In [9]:
df.head()

,book_id,author_id,reader_id,publisher_id,num_pages_norm,book_rating_norm,book_price_norm,2000,2001,2002,...,8,9,10,1,2,3,4,5,6,7
37096,1,10,25885,14,576.88,8.888889,162.994975,1,0,0,...,1,0,0,0,1,0,0,0,0,0
27738,1,25,23452,36,307.88,5.888889,71.994975,0,0,0,...,0,1,0,0,0,0,0,1,0,0
27851,1,226,17207,26,165.88,2.888889,41.994975,0,0,0,...,0,0,0,1,0,0,0,0,0,0
15639,1,194,11178,30,624.88,1.888889,27.994975,0,0,0,...,0,0,0,1,0,0,0,0,0,0
65393,1,380,14465,1,151.88,0.888889,126.994975,1,0,0,...,0,0,1,0,0,0,0,0,0,1
